
# 🦙📄 Basic RAG using Ollama + FAISS

This notebook demonstrates a simple Retrieval-Augmented Generation (RAG) setup using:

- `Ollama` as the backend for LLM (running `llama3.2` or similar)
- `FAISS` for vector storage
- `LangChain` to manage the pipeline

---

## ⚙️ Prerequisites

1. **Download & run Ollama**  
   Make sure Ollama is installed and actively running in the background. You can get it from [https://ollama.com](https://ollama.com).

2. **Load the model you plan to use**  
   This notebook uses the `llama3` model by default. To use it (or any other), run:

   ```bash
   ollama run llama3
   ```

   If you want to switch to another model (e.g., `mistral`, `gemma`, `phi3`), install it first:

   ```bash
   ollama run <model>
   ```

   Then update the model name in the `ChatOllama` function accordingly.


In [19]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama

In [20]:
# Load PDF and split it into chunks
pdf_file = 'sample.pdf'
chunk_size = 1000
chunk_overlap = 200

loader = PyPDFLoader(pdf_file)
documents = loader.load()

# Split the document into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
texts = text_splitter.split_documents(documents)

In [21]:
texts[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Macintosh)', 'creationdate': '2024-06-18T14:09:48-07:00', 'moddate': '2024-06-18T14:10:14-07:00', 'trapped': '/False', 'source': 'sample.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Before using iPhone, review the iPhone User Guide  at  \nsupport.apple.com/guide/iphone .\nSafety and Handling\nSee “Safety, handling, and support” in the iPhone  \nUser Guide .\nExposure to Radio Frequency\nOn iPhone, go to Settings > General > Legal &  \nRegulatory > RF Exposure. Or go to apple.com/  \nlegal/rfexposure .\nBattery and Charging\nAn iPhone battery should only be repaired by a trained \ntechnician to avoid battery damage, which could cause \noverheating, fire, or injury. Batteries should be recycled \nor disposed of separately from household waste and \naccording to local environmental laws and guidelines. For \ninformation about Apple lithium-ion batteries and battery \nservice a

In [22]:
len(texts)

8

In [23]:
# 🔧 Specify the Hugging Face embedding model to use
model_name = "sentence-transformers/all-mpnet-base-v2"

# ⚙️ Model-specific arguments (e.g., run on CPU)
model_kwargs = {'device': 'cpu'}

# 🧮 Embedding behavior settings
# normalize_embeddings=False ensures raw embeddings are used (not L2-normalized)
encode_kwargs = {'normalize_embeddings': False}

# 🤗 Initialize the HuggingFaceEmbeddings class with specified settings
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [24]:
# vectorstore
vectorstore = FAISS.from_documents(
    texts,
    embedding=hf
)

In [25]:
retriever = vectorstore.as_retriever()

In [26]:
# Define the custom prompt template for generation

template = """
You are a customer service agent for a apple mobile company. 
You have been given the following information about the customer query and the context.
Customer Query: {query}
Context: {context}

### Answer: 
The answer should be based on the context provided.
Your task is to answer the customer query based on the context provided. If the question is not related to the context, please say "I don't know or Do Not Answer it just say please ask me question related to Apple Mobiles only".
Do not make up any information or provide any personal opinions or experiences.
Please answer in a friendly and professional manner.
"""

In [27]:
prompt = ChatPromptTemplate.from_messages(
    [
	("system", "You are a helpful assistant."),
	("human", template),
    ]
)

In [ ]:
# 🧠 Load LLM from Ollama (must be running in background)
# In this example we are using the Llama 3.2 model
# Make sure to have the model downloaded and running in the background
llm = ChatOllama(
    model="llama3.2",
    temperature=0,
)

In [29]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [30]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [31]:
# Example 1: Ask a domain-specific question
response = rag_chain.invoke('Is there a warranty on the phone?')
print('📦 Answer:', response)

📦 Answer: Hello! Thank you for reaching out to us about your iPhone warranty. I'd be happy to help.

To confirm, the warranty on your iPhone is subject to certain conditions, including that it may not cause harmful interference and must accept any interference received, including interference that may cause undesired operation.

Regarding your specific question, yes, there is a warranty on your iPhone. Apple warrants the included hardware product and accessories against defects in materials and workmanship for one year from the date of original retail purchase.

If you'd like to make a claim under this warranty, I can guide you through the process. Please note that you may be required to furnish proof of purchase details when making a claim. You can find more information on our warranty terms and conditions at apple.com/legal/warranty or support.apple.com.

Is there anything else I can help with regarding your iPhone warranty?


In [32]:
# Example 2: Ask a general knowledge question
response = rag_chain.invoke('What is GTA 6?')
print('🎮 Answer:', response)

🎮 Answer: Hello! I'd be happy to help you with your query. However, I need to clarify that the context provided seems to be more related to iPhone safety guidelines and precautions rather than answering questions about GTA 6.

To be honest, I don't have any information about GTA 6 in relation to Apple mobiles or iPhones. Could you please ask me a question related to Apple mobiles or iPhones? I'll do my best to assist you with your query.


<!-- Font Awesome CDN (Add in <head> if not already included) -->
<link
  rel="stylesheet" 
  href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.0/css/all.min.css"
/>

<!-- Social Footer Section -->
<div style="
  background-color:rgb(199, 195, 195);
  padding: 40px 30px;
  border-radius: 20px;
  box-shadow: 0 4px 12px rgba(0,0,0,0.08);
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  font-size: 18px;
  max-width: 900px;
  margin: 60px auto 30px;
  text-align: center;
  color: #444;
">
<!-- End of Notebook Note -->
  <h2 style="margin-bottom: 10px;">📘 End of Notebook</h2>
  <p style="color: #666; font-size: 14px;">
    Thank you for exploring! Feel free to connect via the links below.
  </p>

  <!-- Social Icons -->
<div style="
  display: flex;
  gap: 25px;
  align-items: center;
  flex-wrap: wrap;
  justify-content: center;
  margin-bottom: 25px;
">
  <!-- LinkedIn -->
  <a href="https://www.linkedin.com/in/ChiragB254" target="_blank" style="text-decoration: none; color: #0077b5;">
    <i class="fab fa-linkedin fa-lg"></i> LinkedIn
  </a>

  <!-- GitHub -->
  <a href="https://github.com/ChiragB254" target="_blank" style="text-decoration: none; color: #333;">
    <i class="fab fa-github fa-lg"></i> GitHub
  </a>

  <!-- Instagram -->
  <a href="https://www.instagram.com/data.scientist_chirag" target="_blank" style="text-decoration: none; color: #E1306C;">
    <i class="fab fa-instagram fa-lg"></i> Instagram
  </a>

  <!-- Email -->
  <a href="mailto:devchirag27@gmail.com" style="text-decoration: none; color: #D44638;">
    <i class="fas fa-envelope fa-lg"></i> Email
  </a>

  <!-- X (Twitter) -->
  <a href="https://x.com/ChiragB254" target="_blank" style="text-decoration: none; color: #000;">
    <i class="fab fa-x-twitter fa-lg"></i> X.com
  </a>
  </div>

  <p style="font-size: 13px; color: black; font-style: italic; margin-top: 8px;">
    <strong>Made with ❤️ by Chirag Bansal</strong>
  </p>
</div>

